# 🍎 Obstklassifikation - Vorhersage mit trainiertem Modell

Dieses Notebook lädt ein bereits trainiertes Modell und verwendet es für Vorhersagen.

**Vorteile:**
- ⚡ Kein erneutes Training nötig
- 🚀 Schnelle Vorhersagen
- 💾 Nutzt gespeicherte Modelle aus `model_output/`

**Verfügbare Modelle:**
- `best_fruit_classifier_cnn.keras` - Bestes Custom CNN Modell
- `label_mapping_cnn.json` - Zuordnung der Klassen

## 1. Bibliotheken importieren

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import json
from pathlib import Path

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import load_img, img_to_array

print(f"TensorFlow Version: {tf.__version__}")
print(f"GPU verfügbar: {tf.config.list_physical_devices('GPU')}")

## 2. Trainiertes Modell laden

In [ ]:
# Pfade definieren
MODEL_PATH = 'model_output/best_fruit_classifier_cnn.keras'
LABEL_MAPPING_PATH = 'model_output/label_mapping_cnn.json'

# Modell laden
print("Lade trainiertes Modell...")
model = tf.keras.models.load_model(MODEL_PATH)
print(f"✓ Modell geladen von: {MODEL_PATH}")

# Label-Mapping laden
with open(LABEL_MAPPING_PATH, 'r') as f:
    label_mapping = json.load(f)

# Konvertiere String-Keys zu Integer
label_map = {int(k): v for k, v in label_mapping.items()}
print(f"✓ Label-Mapping geladen: {len(label_map)} Klassen")
print(f"\nKlassen: {list(label_map.values())}")

# Modell-Zusammenfassung
print("\n" + "="*60)
print("MODELL-ÜBERSICHT")
print("="*60)
model.summary()

## 3. Vorhersage-Funktion definieren

In [ ]:
def predict_image(model, img_path, label_map, target_size=(100, 100)):
    """
    Macht eine Vorhersage für ein einzelnes Bild.
    
    Args:
        model: Trainiertes Keras-Modell
        img_path: Pfad zum Bild
        label_map: Dictionary mit Index -> Klassenname
        target_size: Zielgröße für das Bild (100x100 für Fruits-360)
    
    Returns:
        predicted_class: Name der vorhergesagten Klasse
        confidence: Konfidenz der Vorhersage (0-100%)
        all_probabilities: Wahrscheinlichkeiten für alle Klassen
    """
    # Bild laden und vorbereiten
    img = load_img(img_path, target_size=target_size)
    img_array = img_to_array(img)
    img_array = img_array / 255.0  # Normalisierung
    img_array = np.expand_dims(img_array, axis=0)  # Batch-Dimension hinzufügen
    
    # Vorhersage
    predictions = model.predict(img_array, verbose=0)
    predicted_index = np.argmax(predictions[0])
    confidence = predictions[0][predicted_index] * 100
    
    predicted_class = label_map[predicted_index]
    
    # Alle Wahrscheinlichkeiten
    all_probs = {label_map[i]: predictions[0][i] * 100 for i in range(len(label_map))}
    
    return predicted_class, confidence, all_probs, img


def display_prediction(img_path, predicted_class, confidence, all_probs, img):
    """
    Zeigt das Bild mit Vorhersage-Ergebnissen an.
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Bild anzeigen
    ax1.imshow(img)
    ax1.axis('off')
    ax1.set_title(f'Vorhersage: {predicted_class}\nKonfidenz: {confidence:.2f}%', 
                  fontsize=14, fontweight='bold')
    
    # Wahrscheinlichkeiten als Balkendiagramm
    classes = list(all_probs.keys())
    probabilities = list(all_probs.values())
    colors = ['green' if c == predicted_class else 'lightblue' for c in classes]
    
    ax2.barh(classes, probabilities, color=colors)
    ax2.set_xlabel('Wahrscheinlichkeit (%)', fontsize=12)
    ax2.set_title('Vorhersage-Wahrscheinlichkeiten', fontsize=14, fontweight='bold')
    ax2.set_xlim(0, 100)
    
    # Werte an den Balken anzeigen
    for i, (cls, prob) in enumerate(zip(classes, probabilities)):
        ax2.text(prob + 1, i, f'{prob:.1f}%', va='center', fontsize=10)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n📁 Bild: {os.path.basename(img_path)}")
    print(f"🎯 Vorhersage: {predicted_class}")
    print(f"✓ Konfidenz: {confidence:.2f}%")

print("✓ Vorhersage-Funktionen definiert")

## 4. Einzelne Vorhersage - Beispiel

Verwenden Sie ein Bild aus dem Test-Datensatz oder laden Sie Ihr eigenes Bild.

In [ ]:
# Beispiel: Vorhersage für ein Testbild
# Passen Sie den Pfad an Ihr Bild an

# Beispiel 1: Bild aus dem Test-Datensatz
test_image_path = r'C:\Projekte\fruits-360-100x100\Test\Apple\0_100.jpg'

# Beispiel 2: Eigenes Bild (auskommentiert)
# test_image_path = 'mein_obst_bild.jpg'

# Prüfen ob Datei existiert
if os.path.exists(test_image_path):
    predicted_class, confidence, all_probs, img = predict_image(
        model, test_image_path, label_map
    )
    display_prediction(test_image_path, predicted_class, confidence, all_probs, img)
else:
    print(f"❌ Bild nicht gefunden: {test_image_path}")
    print("Bitte passen Sie den Pfad an!")

## 5. Mehrere Bilder vorhersagen

Vorhersagen für mehrere Bilder gleichzeitig.

In [ ]:
import glob

# Beispiel: Alle Apfel-Bilder aus dem Test-Ordner
test_folder = r'C:\Projekte\fruits-360-100x100\Test\Apple'

# Oder einen eigenen Ordner verwenden
# test_folder = 'meine_bilder'

if os.path.exists(test_folder):
    # Erste 6 Bilder aus dem Ordner
    image_files = glob.glob(os.path.join(test_folder, '*.jpg'))[:6]
    
    if len(image_files) > 0:
        # Grid für mehrere Bilder
        fig, axes = plt.subplots(2, 3, figsize=(15, 10))
        axes = axes.flatten()
        
        for idx, img_path in enumerate(image_files):
            predicted_class, confidence, all_probs, img = predict_image(
                model, img_path, label_map
            )
            
            axes[idx].imshow(img)
            axes[idx].axis('off')
            axes[idx].set_title(
                f'{predicted_class}\n{confidence:.1f}%',
                fontsize=12,
                fontweight='bold',
                color='green' if confidence > 90 else 'orange'
            )
        
        plt.tight_layout()
        plt.show()
        
        print(f"✓ {len(image_files)} Bilder vorhergesagt")
    else:
        print(f"❌ Keine Bilder im Ordner gefunden: {test_folder}")
else:
    print(f"❌ Ordner nicht gefunden: {test_folder}")
    print("Bitte passen Sie den Pfad an!")

## 6. Eigenes Bild hochladen und vorhersagen

Laden Sie Ihr eigenes Obstbild und lassen Sie es klassifizieren.

In [ ]:
# Geben Sie hier den Pfad zu Ihrem eigenen Bild an
custom_image_path = 'mein_obst.jpg'  # Passen Sie dies an!

if os.path.exists(custom_image_path):
    predicted_class, confidence, all_probs, img = predict_image(
        model, custom_image_path, label_map
    )
    display_prediction(custom_image_path, predicted_class, confidence, all_probs, img)
else:
    print(f"ℹ️ Bild nicht gefunden: {custom_image_path}")
    print("")
    print("So verwenden Sie diese Zelle:")
    print("1. Legen Sie ein Obstbild in den Projektordner")
    print("2. Ändern Sie 'custom_image_path' zum Dateinamen")
    print("3. Führen Sie die Zelle erneut aus")
    print("")
    print("Beispiel: custom_image_path = 'mein_apfel.jpg'")

## 7. Modell-Informationen

Zeigt Details über das geladene Modell.

In [ ]:
print("="*60)
print("MODELL-INFORMATIONEN")
print("="*60)
print(f"Modell-Pfad: {MODEL_PATH}")
print(f"Anzahl Klassen: {len(label_map)}")
print(f"Klassen: {', '.join(label_map.values())}")
print(f"")
print(f"Input Shape: {model.input_shape}")
print(f"Output Shape: {model.output_shape}")
print(f"Anzahl Parameter: {model.count_params():,}")
print(f"Anzahl Layer: {len(model.layers)}")
print("="*60)

## 💡 Tipps zur Verwendung

### Modell wechseln
Falls Sie ein anderes trainiertes Modell verwenden möchten:
```python
MODEL_PATH = 'model_output/custom_cnn_fruit_classifier.keras'
LABEL_MAPPING_PATH = 'model_output/label_mapping.json'
```

### Beste Ergebnisse erzielen
- ✅ Verwenden Sie klare, gut beleuchtete Bilder
- ✅ Das Obst sollte den Großteil des Bildes ausfüllen
- ✅ Einfarbiger Hintergrund funktioniert am besten
- ⚠️ Das Modell wurde auf Fruits-360 trainiert (weißer Hintergrund)

### Verfügbare Obstklassen
Das Modell kann folgende Früchte erkennen:
- 🍎 Apple (Apfel)
- 🍌 Banana (Banane)
- 🍒 Cherry (Kirsche)
- 🥝 Kiwi
- 🍋 Lemon (Zitrone)
- 🍊 Orange
- 🍑 Peach (Pfirsich)
- 🍓 Strawberry (Erdbeere)
- 🍅 Tomato (Tomate)